* 원본 데이터
    * 국내 유통되는 다빈도(처방 및 판매) 경구약제 5,000종을 선정하여 다양한 각도, 조명, 배경에서 촬영하여 이미지데이터 셋을 구축하였고 알약과 카메라 메타정보, 알약 바운딩박스 정보를 이용하여 라벨링데이터 셋을 구축하였음
    * 5000
* 데이터
    
* 이미지 파일명 
    * "k-알약1-알약2-알약3"



In [2]:
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

# 1.데이터불러오기

In [2]:
train_img_dir = "../data/train_images"
train_annots_dir = "../data/train_annotations"
test_img_dir = "../data/test_images"

train_images = sorted([f for f in os.listdir(train_img_dir) if f.endswith(".png")])
train_annots = sorted([f for f in os.listdir(train_annots_dir) if f.endswith(".json")])
test_images = sorted([f for f in os.listdir(test_img_dir) if f.endswith(".png")])

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_images'

#2.데이터 전처리

In [16]:
# .png 가져오기
test_images = sorted([f for f in os.listdir(test_img_dir) if f.endswith(".png")])

# 파일 리스트 출력
print(f"train_imgs: 총 {len(test_images)}개의 png 파일")


# 모든 하위 폴더에서 .json 파일 리스트 가져오기
json_files = []
for root, _, files in os.walk(train_annots_dir):
    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))  # 전체 경로 포함
sorted(json_files)
# 파일 개수 확인 및 출력
print(f"train_annots: 총 {len(json_files)}개의 JSON 파일")


test_images = sorted([f for f in os.listdir(test_img_dir) if f.endswith(".png")])
print(f"test_imgs: 총 {len(test_images)}개의 png 파일")

train_imgs: 총 843개의 png 파일
train_annots: 총 4526개의 JSON 파일
test_imgs: 총 843개의 png 파일


In [ ]:
import os

def print_directory_structure(path, indent=""):
    """주어진 경로의 폴더 구조를 트리 형태로 출력합니다."""
    try:
        items = os.listdir(path)
        for item in items:
            full_path = os.path.join(path, item)
            if os.path.isdir(full_path):
                print(indent + "├── " + item + "/")
                print_directory_structure(full_path, indent + "│   ")
            else:
                print(indent + "├── " + item)
    except FileNotFoundError:
        print(f"경로 '{path}'를 찾을 수 없습니다.")
    except PermissionError:
        print(f"경로 '{path}'에 접근할 권한이 없습니다.")

# 현재 작업 디렉토리의 폴더 구조 출력
print_directory_structure(".")

# 특정 경로의 폴더 구조 출력
# print_directory_structure("/content")

# 3.데이터살펴보기

# 새로운 어노테이션에대한 , 알약 id와의 다른 어노테이션 관계를 eda


In [3]:
train_annots_dir = "../data/train_annotations" 
train_annots = sorted([f for f in os.listdir(train_annots_dir) if f.endswith(".json")])

In [98]:
import pandas as pd
import json
import os

def process_json_files(folder_path):
    """폴더 안의 JSON 파일을 읽어 데이터프레임을 생성합니다."""
    data = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        json_data = json.load(f)
                        if "images" in json_data and len(json_data["images"]) > 0:
                            image_data = json_data["images"][0]
                            annotation_data = json_data["annotations"][0] if "annotations" in json_data and len(json_data["annotations"]) > 0 else {}
                            category_data = json_data["categories"][0] if "categories" in json_data and len(json_data["categories"]) > 0 else {}

                            record = {
                                "supercategory": category_data.get("supercategory"),
                                "category_id": category_data.get("id"),
                                "category_name": category_data.get("name"),
                                "file_path": file_path,
                                "file_name": image_data.get("file_name"),
                                "width": image_data.get("width"),
                                "height": image_data.get("height"),
                                "drug_N": image_data.get("drug_N"),
                                "drug_S": image_data.get("drug_S"),
                                "back_color": image_data.get("back_color"),
                                "drug_dir": image_data.get("drug_dir"),
                                "light_color": image_data.get("light_color"),
                                "camera_la": image_data.get("camera_la"),
                                "camera_lo": image_data.get("camera_lo"),
                                "size": image_data.get("size"),
                                "dl_idx": image_data.get("dl_idx"),
                                "dl_mapping_code": image_data.get("dl_mapping_code"),
                                "dl_name": image_data.get("dl_name"),
                                "dl_name_en": image_data.get("dl_name_en"),
                                "img_key": image_data.get("img_key"),
                                "dl_material": image_data.get("dl_material"),
                                "dl_material_en": image_data.get("dl_material_en"),
                                "dl_custom_shape": image_data.get("dl_custom_shape"),
                                "dl_company": image_data.get("dl_company"),
                                "dl_company_en": image_data.get("dl_company_en"),
                                "item_seq": image_data.get("item_seq"),
                                "di_item_permit_date": image_data.get("di_item_permit_date"),
                                "di_class_no": image_data.get("di_class_no"),
                                "di_etc_otc_code": image_data.get("di_etc_otc_code"),
                                "di_edi_code": image_data.get("di_edi_code"),
                                "chart": image_data.get("chart"),
                                "drug_shape": image_data.get("drug_shape"),
                                "thick": image_data.get("thick"),
                                "leng_long": image_data.get("leng_long"),
                                "leng_short": image_data.get("leng_short"),
                                "print_front": image_data.get("print_front"),
                                "print_back": image_data.get("print_back"),
                                "color_class1": image_data.get("color_class1"),
                                "color_class2": image_data.get("color_class2"),
                                "line_front": image_data.get("line_front"),
                                "line_back": image_data.get("line_back"),
                                "img_regist_ts": image_data.get("img_regist_ts"),
                                "form_code_name": image_data.get("form_code_name"),
                                "mark_code_front_anal": image_data.get("mark_code_front_anal"),
                                "mark_code_back_anal": image_data.get("mark_code_back_anal"),
                                "mark_code_front_img": image_data.get("mark_code_front_img"),
                                "mark_code_back_img": image_data.get("mark_code_back_img"),
                                "mark_code_front": image_data.get("mark_code_front"),
                                "mark_code_back": image_data.get("mark_code_back"),
                                "change_date": image_data.get("change_date"),
                                "id": image_data.get("id"),
                                "area": annotation_data.get("area"),
                                "iscrowd": annotation_data.get("iscrowd"),
                                "bbox": annotation_data.get("bbox"),
                                "category_id": annotation_data.get("category_id"),
                                "ignore": annotation_data.get("ignore"),
                                "segmentation": annotation_data.get("segmentation"),
                                "annotation_id": annotation_data.get("id"),
                                "image_id": annotation_data.get("image_id"),
                            }
                            data.append(record)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

    df = pd.DataFrame(data)
    return df

# JSON 파일이 있는 폴더 경로를 지정하세요.
folder_path = train_annots_dir  # 실제 폴더 경로로 변경
df = process_json_files(folder_path)
df.head() # 데이터프레임의 첫 5개 행을 출력하여 변경사항 확인

,supercategory,category_id,category_name,file_path,file_name,width,height,drug_N,drug_S,back_color,...,mark_code_back,change_date,id,area,iscrowd,bbox,ignore,segmentation,annotation_id,image_id
0,pill,27776,카나브정 60mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_75_000_2...,976,1280,K-027777,정상알약,연회색 배경,...,,20210309,29,37080,0,"[145, 775, 180, 206]",0,[],111,29
1,pill,27776,카나브정 60mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_70_000_2...,976,1280,K-027777,정상알약,연회색 배경,...,,20210309,30,34692,0,"[638, 293, 177, 196]",0,[],109,30
2,pill,27776,카나브정 60mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_90_000_2...,976,1280,K-027777,정상알약,연회색 배경,...,,20210309,28,35223,0,"[176, 812, 177, 199]",0,[],110,28
3,pill,25366,자누메트정 50/850mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_75_000_2...,976,1280,K-025367,정상알약,연회색 배경,...,,20181112,29,178253,0,"[9, 49, 397, 449]",0,[],945,29
4,pill,25366,자누메트정 50/850mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_70_000_2...,976,1280,K-025367,정상알약,연회색 배경,...,,20181112,30,196729,0,"[566, 726, 409, 481]",0,[],105,30


In [99]:
import json
import os
import re
import pandas as pd

def analyze_json_files(folder_path):
    """주어진 폴더의 JSON 파일을 분석하여 알약 개수와 어노테이션 개수를 비교합니다."""
    analysis_results = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'r') as f:
                data = json.load(f)

            # 파일 이름에서 알약 번호 추출
            pill_numbers = re.findall(r'\d{6}', filename)
            expected_pill_count = len(pill_numbers)

            # 어노테이션 개수 확인
            actual_annotation_count = len(data['annotations'])

            analysis_results.append({
                'filename': filename,
                'expected_pill_count': expected_pill_count,
                'actual_annotation_count': actual_annotation_count
            })

    return pd.DataFrame(analysis_results)

# JSON 파일이 있는 폴더 경로를 지정하세요.
folder_path = "../data/train_annots_modify"
results_df = analyze_json_files(folder_path)

# 차이 계산
results_df['difference'] = results_df['expected_pill_count'] - results_df['actual_annotation_count']

# 불일치 레코드만 출력
mismatch_df = results_df[results_df['difference'] != 0]
print("불일치 레코드:")
mismatch_df

불일치 레코드:


,filename,expected_pill_count,actual_annotation_count,difference
477,K-003544-004543-016548-023203_0_2_0_2_90_000_2...,4,3,1


In [ ]:
df.isna().sum().sort_values(ascending=False)  # 결측치 확인 >> 많지않다고 판단했습니다. 172 / 1479

mark_code_back_anal     172
mark_code_front_img     172
print_front             172
print_back              172
color_class1            172
color_class2            172
line_front              172
img_key                 172
line_back               172
img_regist_ts           172
form_code_name          172
mark_code_front_anal    172
chart                   172
drug_shape              172
mark_code_back_img      172
mark_code_front         172
mark_code_back          172
change_date             172
id                        0
area                      0
iscrowd                   0
bbox                      0
ignore                    0
segmentation              0
annotation_id             0
leng_short                0
leng_long                 0
thick                     0
supercategory             0
di_edi_code               0
category_id               0
di_etc_otc_code           0
category_name             0
file_path                 0
file_name                 0
width               

제공된 JSON 파일은 이미지 데이터셋의 주석(annotation) 정보를 담고 있습니다. 각 섹션을 나누어 자세히 설명해 드리겠습니다.

**1. `images` 섹션**

* 이 섹션은 이미지 파일에 대한 정보를 담고 있습니다.
* `file_name`: 이미지 파일의 이름입니다.
* `width`: 이미지의 가로 픽셀 수입니다.
* `height`: 이미지의 세로 픽셀 수입니다.
* `imgfile`: 이미지 파일 이름과 동일합니다.
* `drug_N`: 약품 코드입니다.  **        Value: ['K-027777', 'K-025367', 'K-003483'
* `drug_S`: 약품 상태 (정상알약만 존재)  **
* `back_color`: 배경 색상입니다.
* `drug_dir`: 약품 방향 (앞면)입니다.
* `light_color`: 조명 색상입니다.
* `camera_la`: 카메라 각도 (latitude)   Value: ['75', '70', '90']
* `camera_lo`: 카메라 각도 (longitude)  Value: ['0']
* `size`: 이미지 크기입니다.
* `dl_idx`: 데이터 레이블 인덱스입니다.
* `dl_mapping_code`: 데이터 레이블 매핑 코드입니다. Value: ['K-027777', 'K-025367', 
* `dl_name`: 약품 이름입니다.  ** Value: ['카나브정 60mg', '자누메트정 50/850mg', 
* `dl_name_en`: 약품 영문 이름입니다.
* `img_key`: 이미지 URL입니다.
* `dl_material`: 약품 성분입니다.
* `dl_material_en`: 약품 영문 성분입니다.
* `dl_custom_shape`: 약품 형태입니다.
* `dl_company`: 제조사입니다.
* `dl_company_en`: 제조사 영문 이름입니다.
* `di_company_mf`: 제조사 코드입니다.
* `di_company_mf_en`: 제조사 영문 코드입니다.
* `item_seq`: 품목 일련번호입니다.
* `di_item_permit_date`: 품목 허가일입니다.
* `di_class_no`: 약품 분류 번호입니다.
* `di_etc_otc_code`: 전문/일반 의약품 코드입니다.
* `di_edi_code`: EDI 코드입니다.
* `chart`: 약품 설명입니다.
* `drug_shape`: 약품 모양입니다.  **
* `thick`: 약품 두께입니다.  **
* `leng_long`: 약품 긴쪽 길이입니다.  **
* `leng_short`: 약품 짧은쪽 길이입니다.  **
* `print_front`: 앞면 각인입니다.  **
* `print_back`: 뒷면 각인입니다.  **
* `color_class1`: 색상 분류1입니다.  **
* `color_class2`: 색상 분류2입니다.  **
* `line_front`: 앞면 분할선입니다.
* `line_back`: 뒷면 분할선입니다.
* `img_regist_ts`: 이미지 등록 시간입니다.
* `form_code_name`: 제형 코드입니다.  **
* `mark_code_front_anal`: 앞면 마크 분석 코드입니다.
* `mark_code_back_anal`: 뒷면 마크 분석 코드입니다.
* `mark_code_front_img`: 앞면 마크 이미지 코드입니다.
* `mark_code_back_img`: 뒷면 마크 이미지 코드입니다.
* `mark_code_front`: 앞면 마크 코드입니다.
* `mark_code_back`: 뒷면 마크 코드입니다.
* `change_date`: 변경일입니다.
* `id`: 이미지 ID입니다.

**2. `type` 섹션**

* 주석 유형을 나타냅니다. 여기서는 "instances"로, 객체 인스턴스에 대한 주석임을 의미합니다.

**3. `annotations` 섹션**

* 이 섹션은 이미지 내 객체에 대한 주석 정보를 담고 있습니다.
* `area`: 객체 영역의 넓이입니다.
* `iscrowd`: 객체가 군집(crowd)인지 여부입니다.
* `bbox`: 객체의 바운딩 박스 좌표 [x, y, width, height]입니다.
* `category_id`: 객체의 카테고리 ID입니다.
* `ignore`: 객체를 무시할지 여부입니다.
* `segmentation`: 객체의 분할 정보입니다.
* `id`: 주석 ID입니다.
* `image_id`: 주석이 연결된 이미지 ID입니다.

**4. `categories` 섹션**

* 이 섹션은 객체 카테고리에 대한 정보를 담고 있습니다.
* `supercategory`: 상위 카테고리입니다.
* `id`: 카테고리 ID입니다.
* `name`: 카테고리 이름입니다.

이 JSON 파일은 이미지 데이터셋의 객체 감지(object detection) 또는 분할(segmentation) 모델 학습에 사용될 수 있습니다.


In [86]:
df[:3]

,supercategory,category_id,category_name,file_path,file_name,width,height,drug_N,drug_S,back_color,...,mark_code_back,change_date,id,area,iscrowd,bbox,ignore,segmentation,annotation_id,image_id
0,pill,27776,카나브정 60mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_75_000_2...,976,1280,K-027777,정상알약,연회색 배경,...,,20210309,29,37080,0,"[145, 775, 180, 206]",0,[],111,29
1,pill,27776,카나브정 60mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_70_000_2...,976,1280,K-027777,정상알약,연회색 배경,...,,20210309,30,34692,0,"[638, 293, 177, 196]",0,[],109,30
2,pill,27776,카나브정 60mg,../data/train_annotations/K-003483-025367-0277...,K-003483-025367-027777-035206_0_2_0_2_90_000_2...,976,1280,K-027777,정상알약,연회색 배경,...,,20210309,28,35223,0,"[176, 812, 177, 199]",0,[],110,28


In [88]:
def get_unique_values_dict(df):
    """데이터프레임의 모든 칼럼에 대해 유니크 값을 딕셔너리로 저장합니다."""
    unique_values_dict = {}
    for column in df.columns:
        # 리스트를 문자열로 변환하여 저장
        unique_values_dict[column] = df[column].astype(str).unique().tolist()
    return unique_values_dict

unique_values_dict = get_unique_values_dict(df)


In [106]:
def print_dict_first_10(unique_values_dict):
    """딕셔너리의 키와 값(리스트)에 대해 처음 10개 요소만 출력합니다."""
    for key, value in unique_values_dict.items():
        print(f"{key:<{15}}", end=" \t")
        if isinstance(value, list):
            print(f"Value: {value[:10]}")
        else:
            print(f"Value: {value}")


unique_10 = print_dict_first_10(unique_values_dict)
unique_10

supercategory   	Value: ['pill']
category_id     	Value: ['27776', '25366', '3482', '35205', '36636', '27732', '28762', '27925', '1899', '24849']
category_name   	Value: ['카나브정 60mg', '자누메트정 50/850mg', '기넥신에프정(은행엽엑스)(수출용)', '아토젯정 10/40mg', '로수젯정10/5밀리그램', '트윈스타정 40/5mg', '트라젠타정(리나글립틴)', '울트라셋이알서방정', '보령부스파정 5mg', '놀텍정 10mg']
file_path       	Value: ['../data/train_annotations/K-003483-025367-027777-035206_json/K-027777/K-003483-025367-027777-035206_0_2_0_2_75_000_200.json', '../data/train_annotations/K-003483-025367-027777-035206_json/K-027777/K-003483-025367-027777-035206_0_2_0_2_70_000_200.json', '../data/train_annotations/K-003483-025367-027777-035206_json/K-027777/K-003483-025367-027777-035206_0_2_0_2_90_000_200.json', '../data/train_annotations/K-003483-025367-027777-035206_json/K-025367/K-003483-025367-027777-035206_0_2_0_2_75_000_200.json', '../data/train_annotations/K-003483-025367-027777-035206_json/K-025367/K-003483-025367-027777-035206_0_2_0_2_70_000_200.json', '../data/trai

In [108]:
'''dataframe으로 보기'''

def join_and_create_dataframe(unique_values_dict):
    """딕셔너리의 리스트 값을 쉼표로 join하고 데이터프레임을 생성합니다."""
    joined_data = {}
    for key, value in unique_values_dict.items():
        if isinstance(value, list):
            joined_data[key] = ', '.join(map(str, value))
        else:
            joined_data[key] = str(value)

    return pd.DataFrame(joined_data, index=[0]).T

view = join_and_create_dataframe(unique_values_dict)
view

,0
supercategory,pill
category_id,"27776, 25366, 3482, 35205, 36636, 27732, 28762..."
category_name,"카나브정 60mg, 자누메트정 50/850mg, 기넥신에프정(은행엽엑스)(수출용),..."
file_path,../data/train_annotations/K-003483-025367-0277...
file_name,K-003483-025367-027777-035206_0_2_0_2_75_000_2...
width,976
height,1280
drug_N,"K-027777, K-025367, K-003483, K-035206, K-0366..."
drug_S,정상알약
back_color,연회색 배경


In [107]:
selected_keys = [
    'drug_N', 'drug_S', 'camera_la', 'camera_lo', 'dl_mapping_code', 'dl_name',
    'di_class_no', 'di_etc_otc_code', 'di_edi_code', 'chart', 'drug_shape',
    'thick', 'leng_long', 'leng_short', 'print_front', 'print_back',
    'color_class1', 'color_class2', 'form_code_name', 'mark_code_front_anal',
    'mark_code_back_anal', 'mark_code_front_img', 'mark_code_back_img',
    'mark_code_front', 'mark_code_back', 'change_date', 'id'
]

for key in selected_keys:
    print(f"{key:<{15}}", end=" \t")
    value = unique_values_dict[key]
    if isinstance(value, list):
        print(f"Value: {value[:10]}")
    else:
        print(f"Value: {value}")

drug_N          	Value: ['K-027777', 'K-025367', 'K-003483', 'K-035206', 'K-036637', 'K-027733', 'K-028763', 'K-027926', 'K-001900', 'K-024850']
drug_S          	Value: ['정상알약']
camera_la       	Value: ['75', '70', '90']
camera_lo       	Value: ['0']
dl_mapping_code 	Value: ['K-027777', 'K-025367', 'K-003483', 'K-035206', 'K-036637', 'K-027733', 'K-028763', 'K-027926', 'K-001900', 'K-024850']
dl_name         	Value: ['카나브정 60mg', '자누메트정 50/850mg', '기넥신에프정(은행엽엑스)(수출용)', '아토젯정 10/40mg', '로수젯정10/5밀리그램', '트윈스타정 40/5mg', '트라젠타정(리나글립틴)', '울트라셋이알서방정', '보령부스파정 5mg', '놀텍정 10mg']
di_class_no     	Value: ['[02140]혈압강하제', '[03960]당뇨병용제', '[02190]기타의 순환계용약', '[02180]동맥경화용제', '[01140]해열.진통.소염제', '[01170]정신신경용제', '[02320]소화성궤양용제', '[01130]항전간제', '[02590]기타의 비뇨생식기관 및 항문용약', '[01190]기타의 중추신경용약']
di_etc_otc_code 	Value: ['전문의약품', '일반의약품']
di_edi_code     	Value: ['641905450', '655500760,E09060771', '644700130,A18901361', '655501870', '643507280', '653501270', '653501380', '646901630', '641901280,A093023